In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

from process import preprocess_all_files

data = preprocess_all_files()

# Extract X and Y from the data
X_data = [item[0] for item in data]
Y_data = [item[1] for item in data]

print(type(X_data))
print(type(Y_data))

# Pad sequences to a fixed length (swing data length is different)
max_sequence_length = 999 
X_data = pad_sequences(X_data, maxlen=max_sequence_length, padding='post', dtype='float32')

# Convert lists to numpy arrays
X_data = np.array(X_data)
Y_data = np.array(Y_data)

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

print("Input data is of shape {X_train.shape}")
print("Output data is of shape {Y_train.shape}")

# Define model
model = Sequential()

model.add(LSTM(64, input_shape=(999, 7), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(10, activation='linear'))

# Compile modle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2) 

# Evaluate model
score = model.evaluate(X_test, Y_test)
print(score)

tf.saved_model.save(model, r"C:\Users\James\Documents\GitHub\IMU_Golf_ML\5.swingmapping")

<class 'list'>
<class 'list'>
Input data is of shape {X_train.shape}
Output data is of shape {Y_train.shape}
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 999, 64)           18432     
                                                                 
 dropout_29 (Dropout)        (None, 999, 64)           0         
                                                                 
 lstm_31 (LSTM)              (None, 999, 64)           33024     
                                                                 
 dropout_30 (Dropout)        (None, 999, 64)           0         
                                                                 
 lstm_32 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_31 (Dropout)        (None, 64)                0         
          

INFO:tensorflow:Assets written to: C:\Users\James\Documents\GitHub\IMU_Golf_ML\5.swingmapping\assets
